### TP3 - 2

2. O seguinte sistema dinâmico denota 4 inversores ($\,A, B, C, D\,$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.


<img src="inversores.png" alt="autómato"/>


   1. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório.
   2. Cada inversor é regido pelas seguintes transformações 
  
     $$ \mathbf{invert}\mathtt(in,out) $$
     $$x \gets \mathsf{read}(\mathtt{in})$$
     $$s \gets \neg x\;\;\|\;\; s\gets s\oplus x$$             
     $$\mathsf{write}(\mathtt{out},s)$$
     
        iii. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicializarão do sistema.
        
        iii. O estado do sistema é um duplo definido pelos 4 bits $s$, e é inicializado com um vetor aleatório em $\{0,1\}^4\;$.
        
        iv. O sistema termina em ERRO quando o estado do sistema for $\,(0,0,0,0)\,$.


1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $\,n\,$ estados.
2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import itertools 
from random import randint

### Função genState bla bla bla

In [2]:
# vars = ['pc', s']
def genState(vars,s,i,n):
    state = {}
    state['pc'] = Symbol('pc'+'_'+s+str(i), BVType(n))
    for j in range(1, 5):
        for v in vars[1:]:
            if v not in state:
                state[v] = {}
            
            state[v][j] = Symbol(v+str(j)+'_'+s+str(i), BVType(n))
            
    #print(state)
    
            
    return state

#### Função init(state):

    state - Dicionário de variáveis de estado


   A função `init` tem como objetivo devolver um predicado do Solver que testa se é um possível estado inicial do programa, através do `state`, um dicionário de variáveis.

In [3]:
def init(state, n):
    random_state = (BV(randint(0,1), n), BV(randint(0,1), n), BV(randint(0,1), n), BV(randint(0,1), n)) 
    

    s_list = []
    for v in state['s']:
        s_list.append( Equals(state['s'][v], random_state[v-1]) )
    
    return And(
        Equals(state['pc'], BV(1, n)),
        And(s_list)
    )

#### Função trans(curr, prox):

    curr - Estado das variáveis no momento atual
    prox - Estado das variáveis no momento da próxima iteração
    
   A função `trans` tem como objetivo devolver um predicado do Solver, através dos três estados disponíveis, que teste se é possível transitar entre os estados possíveis.

In [4]:
def error(state, n):
    return Or( 
        Equals( Or([ BVOr(state['s'][i], BVZero(n)) for i in range(1,5)]) )
    )


def trans(curr, prox, n):
    zero_state = Equals( Or([ BVOr(curr['s'][i], BVZero(n)) for i in range(1,5)]), BVZero(n))
    
    # prox_x = prox_s
    # prox_s = Not(curr[x])  V Xor(curr[s], curr[x])  
    t1_2 = And(
        Equals(curr['pc'], BVOne(n)),
            
        Or(
            And(BVNot(zero_state), Equals(prox['pc'], BV(2, n))),
            And(zero_state, Equals(prox['pc'], BV(5, n)))
        ),
        
        # o x é o s do inversor anterior
        # prox_s = Not(curr[x])  V Xor(curr[s], curr[x])
        Equals(prox['s'][1], Or(BVNot(curr['s'][4]), BVXor(curr['s'][1], curr['s'][4]) )), 
        
        And([ Equals(prox['s'][i], curr['s'][i]) for i in range(2,5)])
        
    )  
    
    t2_3 = And(
        Equals(curr['pc'], BV(2, n)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], BV(3, n))),
            And(zero_state, Equals(prox['pc'], BV(5, n)))
        ),
        
        Equals(prox['s'][2], Or(BVNot(curr['s'][1]), BVXor(curr['s'][2], curr['s'][1]) )),
        
        Equals(prox['s'][1], curr['s'][1]),
        And([Equals(prox['s'][i], curr['s'][i]) for i in range(3,5)]) 
    )
    
    t3_4 = And(
        Equals(curr['pc'], BV(3, n)),
            
        Or(
            And(BVNot(zero_state), Equals(prox['pc'], BV(4, n))),
            And(zero_state, Equals(prox['pc'], BV(5, n)))
        ),
        
        Equals(prox['s'][3], Or(BVNot(curr['s'][2]), BVXor(curr['s'][3], curr['s'][2]) )),
        
        And([Equals(prox['s'][i], curr['s'][i]) for i in range(1,3)]),
        Equals(prox['s'][4], curr['s'][4])
    )
    
    t4_1 = And(
        Equals(curr['pc'], BV(4, n)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], BV(1, n))),
            And(zero_state, Equals(prox['pc'], BV(5, n)))
        ),
        
        Equals(prox['s'][4], Or(BVNot(curr['s'][3]), BVXor(curr['s'][4], curr['s'][3]) )),
        
        And([Equals(prox['s'][i], curr['s'][i]) for i in range(1,4)])
    )
    
    
    error = And(
        Equals(curr['pc'], BV(5, n)),
        Equals(prox['pc'], curr['pc']),
        zero_state,
        And( [Equals(prox['s'][i], curr['s'][i]) for i in range(1,5)])
    )
    
    return Or(t1_2, t2_3, t3_4, t4_1, error)
    

#### Função gera_traco(declare, init, trans, k, n, a, b)

    declare - Variáveis de estado
    init - Condições para o estado inicial
    trans - Função transição
    k - Valor do traço


A função `gera_traco` tem como objetivo imprimir o valor das variáveis à medida que vão percorrendo os estados, através das variáveis do estado e de um predicado que testa se um estado é inicial.

In [5]:
def gera_traco(vars, init, trans, error, n, k):
    with Solver(name="z3") as s:
    
        X = [genState(vars,'X',i, n) for i in range(k+1)]
        #print(X)
        I = init(X[0], n)
        print(I)
        Tks = [ trans(X[i],X[i+1],n) for i in range(k) ]
        #print(Tks)
    
        if s.solve([I,And(Tks)]):
            for i in range(k):
                print("Passo:",i)
                for v in X[i]:
                    if v == 'pc':
                        print("Estado: ",s.get_value(X[i][v]))
                    else:
                        for k in range(1,5):
                            print("          ",v,'=',s.get_value(X[i][v][k]))
                print("----------------")
        else:
            print("unsat")
        
        
gera_traco(['pc', 's'], init, trans, error, 5, 20)

((pc_X0 = 1_5) & ((s1_X0 = 1_5) & (s2_X0 = 0_5) & (s3_X0 = 0_5) & (s4_X0 = 0_5)))


AttributeError: 'list' object has no attribute 'bv_width'